In [1]:
import numpy as np
from scipy import signal
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit, cross_val_score
import csv
import pickle
import datetime

In [3]:
# set path to datasets
savetag='_exp2'

datapath = '/home/jingyan/Documents/ME499-WinterProject/mindwave/src/motor_cortex_ml/data/'
csvname =datapath + 'record'+savetag+'.csv'
# csvname =datapath +'record_20rfft.csv'
data = np.loadtxt(csvname,delimiter = ',')
savedsp=datapath+'dsp'+savetag+'.csv'
# get input/output pairs
x = data[:,:-1]
y = data[:,-1]
channel_num=8
chunk=20 #matches self.recordsize in mindwave_record.py
fs=250
band=(7,13)

def bandpass(start,stop,data,fs):
    bp_Hz = np.array([start, stop])
    b, a = signal.butter(5, bp_Hz / (fs / 2.0), btype='bandpass')
    return signal.lfilter(b, a, data, axis=0)

In [5]:
for k in range(channel_num):
    locals()["channel"+str(k+1)]=x[:,k]
#Divide datas to chunks, each chunk coorespond to specific channel with specific y value.
#Total chunk number = channel_num * record_size(chunk) *4
_total_chunk=channel_num*chunk*4
for i in np.unique(y):
    i=int(i)
    locals()['ind_'+str(i+1)]=np.where(y==i)[0] ##left=-1,(ind0) ; right=1,(ind2); focus=0,(ind1);
   
    locals()['ind'+str(i+1)]=[locals()['ind_'+str(int(i+1))][0]]
    for j in range(len(locals()['ind_'+str(i+1)])-1):  
        if (locals()['ind_'+str(i+1)])[j]+1!=(locals()['ind_'+str(i+1)])[j+1]:
            locals()['ind'+str(i+1)].append((locals()['ind_'+str(i+1)])[j])
        if j>1 and (locals()['ind_'+str(i+1)])[j]-1!=(locals()['ind_'+str(i+1)])[j-1]:
            (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[j])
    (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[-1])
    if i!=0 and 2*chunk!=len((locals()['ind'+str(i+1)])):
        print("error: chunk size and interval limit does not match. flag number:",i)
    elif i==0 and 4*chunk!=len((locals()['ind'+str(i+1)])):
        print("error: chunk size and interval limit does not match. flag number: ",i)
    for k in range(channel_num):
        for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]=locals()['channel'+str(k+1)][(locals()['ind'+str(i+1)])[2*g]:(locals()['ind'+str(i+1)])[2*g+1]+1]
        

(100000, 8)